In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/05/16 10:52:30 WARN Utils: Your hostname, pclab028 resolves to a loopback address: 127.0.1.1; using 147.96.132.245 instead (on interface eno1)
23/05/16 10:52:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 10:52:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/16 10:52:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import matplotlib.pyplot as plt
import pandas as pd

### Veamos qué podemos hacer con las estaciones. Parece que se nos proporciona un archivo .geo con las localizaiciones. Así que tocará cargarlo de alguna manera y añadirlo al dataframe con su localización.

### 1) GEOPANDAS

In [97]:
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 35.1 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.5 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.0 MB/s eta 0:00:000m eta 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [98]:
import geopandas as gpd
# import matplotlib.pyplot as plt

In [100]:
archivo_geo = "datos/bases_bicimad.geo"
datos_geo = gpd.read_file(archivo_geo)

DriverError: unsupported driver: 'GeoRSS'

### 2) XML

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

In [5]:
# Ruta al archivo .geo
archivo_geo = "datos/bases_bicimad.geo"

# Leer el archivo .geo en formato XML
tree = ET.parse(archivo_geo)
root = tree.getroot()

# Obtener los datos de ubicación de las estaciones
estaciones = []
for item in root.iter('item'):
    latitud = item.find('.//{http://www.georss.org/georss}point').text.split()[0]
    longitud = item.find('.//{http://www.georss.org/georss}point').text.split()[1]
    nombre = item.find('.//{http://www.georss.org/georss}title').text
    estaciones.append({'Nombre': nombre, 'Latitud': latitud, 'Longitud': longitud})

# Crear un DataFrame con los datos
df_estaciones = pd.DataFrame(estaciones)

# Mostrar el DataFrame resultante
print(df_estaciones.head())

Empty DataFrame
Columns: []
Index: []


### 3) KML

In [1]:
!pip install pykml

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pykml import parser

# Ruta al archivo KMZ
archivo_kmz = "datos/Bases_de_BiciMad.kmz"

# Leer el archivo KMZ
with open(archivo_kmz, 'rb') as f:
    kmz = parser.parse(f)

# Extraer el archivo KML del KMZ
kml = kmz.getroot()

# Acceder a los datos del KML
# Por ejemplo, obtener las coordenadas de los puntos
coordenadas = []
placemarks = kml.Document.Folder.Placemark
for placemark in placemarks:
    coordenadas.append(placemark.Point.coordinates.text)

# Mostrar las coordenadas obtenidas
for coordenada in coordenadas:
    print(coordenada)


XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (Bases_de_BiciMad.kmz, line 1)

### 4) PANDAS

Vale, me he vuelto loco y teníamos un .xls por algún lado. Bastante más fácil de leer, podemos con pandas tal cual o con spark (es pequeño por lo que lo leeré con pandas y luego cuando lo 'limpie' lo paso a spark con una función.

In [6]:
import pandas as pd

In [10]:
geo = pd.read_excel('datos/bases_bicimad.xls')

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [11]:
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01


In [12]:
geo = pd.read_excel('datos/bases_bicimad.xls')

In [13]:
geo.head()

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion
0,001 a,440443.61,4474290.65,43803,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2"
1,001 b,440480.56,4474301.74,43867,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",6,BiciMAD,30,-3.701564,40.417213,"ALCALA, CALLE, DE, 6"
2,2,440134.83,4474678.23,41813,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1"
3,3,440012.98,4475760.68,41813,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2"
4,4,440396.40,4475565.36,41813,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3"


BASTANTE MÁS FÁCIL!! Y TENEMOS TODA LA INFO QUE QUERÍAMOS...

In [21]:
print("Number of columns:", geo.columns.size)
print("Number of rows:", geo.index.size)

Number of columns: 13
Number of rows: 269


Puesto que tenemos más datos de los que pensaba, podemos distinguir por barrios o distritos, aunque lo del KMEANs creo que quedaría bien (aunque no sea necesario).

In [29]:
len(set(geo['Barrio'].to_list()))

60

Si distinguimos por barrios, pasaríamos a tener en vez de 269 estaciones, tener 60 grupos de ellas. Dividimos casi por 5, siguen siendo muchas pero ya reducimos bastantes.